In [3]:
from scipy import stats
import time
import requests
from bs4 import BeautifulSoup
import threading
import pandas as pd
import tqdm
import lightgbm as lgb
from sklearn.preprocessing import StandardScaler, PowerTransformer
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

base_url = 'https://www.basketball-reference.com/'
day_scores_base_url = 'https://www.basketball-reference.com/boxscores/?month={month}&day={day}&year={year}'
data_path = r'/media/td/Samsung_T5/sports/nba'
db_name = 'nba_db'
box_score_link_table_name = 'boxscore_links'

box_score_details_table_name = 'boxscore_details'
processed_team_data_table_name = 'processed_team_data'
player_detail_table_name = 'player_details'
processed_player_data_table_name = 'processed_player_data'
combined_feature_file_data_table_name = 'combined_feature_file'

target = 'win'

date_record_pickle_file_name = 'scraped_dates'
box_score_record_pickle_file_name = 'scraped_games'
max_tries = 5
file_lock = threading.Lock()

starting_rating = 1000
rating_k_factor = 100
rating_floor = 100
rating_ceiling = 10000
rating_d = 1000
k_min_sensitivity = 1

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


In [17]:
team_data = pd.read_csv('{data_path}/{db_name}.csv'.format(data_path=data_path,db_name=box_score_details_table_name),sep='|', low_memory=False)
player_data = pd.read_csv('{data_path}/{db_name}.csv'.format(data_path=data_path,db_name=player_detail_table_name), sep='|', low_memory=False)

team_data.shape, player_data.shape

((2, 45), (26, 48))

In [18]:
team_data.head()

,team_tag,team_link,team_name,opponent_tag,opponent_link,opponent_name,location,win,year,month,day,mp,fg,fga,fg_pct,fg3,fg3a,fg3_pct,ft,fta,ft_pct,orb,drb,trb,ast,stl,blk,tov,pf,pts,plus_minus,ts_pct,efg_pct,fg3a_per_fga_pct,fta_per_fga_pct,orb_pct,drb_pct,trb_pct,ast_pct,stl_pct,blk_pct,tov_pct,usg_pct,off_rtg,def_rtg
0,tor,https://www.basketball-reference.com//teams/TO...,Toronto Raptors,gsw,https://www.basketball-reference.com//teams/GS...,Golden State Warriors,"Oracle Arena, Oakland, California",1,2019,6,13,240,39,82,0.476,13,33,0.394,23,29,0.793,11,28,39,25,8,2,12,23,114,NaN,0.602,0.555,0.402,0.354,26.2,71.8,48.1,64.1,8.5,4.1,11.2,100.0,120.5,116.3
1,gsw,https://www.basketball-reference.com//teams/GS...,Golden State Warriors,tor,https://www.basketball-reference.com//teams/TO...,Toronto Raptors,"Oracle Arena, Oakland, California",0,2019,6,13,240,39,80,0.488,11,31,0.355,21,30,0.700,11,31,42,28,9,6,16,23,110,NaN,0.590,0.556,0.388,0.375,28.2,73.8,51.9,71.8,9.5,12.2,14.7,100.0,116.3,120.5


In [19]:
player_data.head()

,ast,ast_pct,blk,blk_pct,day,def_rtg,drb,drb_pct,efg_pct,fg,fg3,fg3_pct,fg3a,fg3a_per_fga_pct,fg_pct,fga,ft,ft_pct,fta,fta_per_fga_pct,location,month,mp,off_rtg,opponent_link,opponent_name,opponent_tag,orb,orb_pct,pf,player_link,player_name,plus_minus,pts,reason,stl,stl_pct,team_link,team_name,team_tag,tov,tov_pct,trb,trb_pct,ts_pct,usg_pct,win,year
0,3.0,10.9,1.0,2.1,13,115.0,8.0,21.3,0.676,10.0,3.0,0.500,6.0,0.353,0.588,17.0,3.0,0.750,4.0,0.235,"Oracle Arena, Oakland, California",6,46:10,132.0,https://www.basketball-reference.com//teams/GS...,Golden State Warriors,gsw,2.0,5.0,2.0,https://www.basketball-reference.com//players/...,Pascal Siakam,2.0,26.0,NaN,1.0,1.1,https://www.basketball-reference.com//teams/TO...,Toronto Raptors,tor,2.0,9.6,10.0,12.8,0.693,20.2,1,2019
1,10.0,40.2,0.0,0.0,13,112.0,5.0,14.8,0.688,9.0,4.0,0.571,7.0,0.438,0.563,16.0,4.0,0.667,6.0,0.375,"Oracle Arena, Oakland, California",6,41:42,137.0,https://www.basketball-reference.com//teams/GS...,Golden State Warriors,gsw,2.0,5.5,5.0,https://www.basketball-reference.com//players/...,Kyle Lowry,16.0,26.0,NaN,3.0,3.7,https://www.basketball-reference.com//teams/TO...,Toronto Raptors,tor,3.0,13.9,7.0,9.9,0.697,23.3,1,2019
2,3.0,11.4,1.0,2.4,13,113.0,5.0,15.0,0.469,7.0,1.0,0.200,5.0,0.313,0.438,16.0,7.0,0.875,8.0,0.500,"Oracle Arena, Oakland, California",6,41:05,116.0,https://www.basketball-reference.com//teams/GS...,Golden State Warriors,gsw,1.0,2.8,4.0,https://www.basketball-reference.com//players/...,Kawhi Leonard,-2.0,22.0,NaN,2.0,2.5,https://www.basketball-reference.com//teams/TO...,Toronto Raptors,tor,2.0,9.3,6.0,8.7,0.564,23.6,1,2019
3,4.0,18.5,0.0,0.0,13,116.0,6.0,27.8,0.000,0.0,0.0,0.000,2.0,0.400,0.000,5.0,3.0,0.750,4.0,0.800,"Oracle Arena, Oakland, California",6,26:34,93.0,https://www.basketball-reference.com//teams/GS...,Golden State Warriors,gsw,3.0,12.9,4.0,https://www.basketball-reference.com//players/...,Marc Gasol,-7.0,3.0,NaN,0.0,0.0,https://www.basketball-reference.com//teams/TO...,Toronto Raptors,tor,1.0,12.9,9.0,20.1,0.222,13.1,1,2019
4,3.0,20.8,0.0,0.0,13,116.0,1.0,6.9,NaN,0.0,0.0,NaN,0.0,NaN,NaN,0.0,0.0,NaN,0.0,NaN,"Oracle Arena, Oakland, California",6,17:43,102.0,https://www.basketball-reference.com//teams/GS...,Golden State Warriors,gsw,0.0,0.0,1.0,https://www.basketball-reference.com//players/...,Danny Green,7.0,0.0,NaN,1.0,2.9,https://www.basketball-reference.com//teams/TO...,Toronto Raptors,tor,1.0,100.0,1.0,3.3,NaN,2.5,1,2019


- Negatively correlated with win
- High stat def rating is highly correlated with losing (.51). Teams under pressure don't win that game.
- Personal fouls also correlated with losing (.13)
- Turnovers - (investigate, not intuitive)
- 

In [5]:
team_data_corr = team_data.corr()
team_data_corr = team_data_corr[(team_data_corr['win'] > .2)|(team_data_corr['win'] < -.08)]
team_data_corr.sort_values('win')[['win']]

,win
stat_def_rtg,-0.520501
stat_tov,-0.111793
stat_tov_pct,-0.106198
stat_pf,-0.090094
stat_fg3,0.262046
stat_trb,0.287934
stat_ast,0.303615
stat_fg3_pct,0.330260
stat_drb,0.362345
stat_fg,0.379037


In [6]:
player_data_corr = player_data.corr()
player_data_corr = player_data_corr[(player_data_corr['win'] > .05)|(player_data_corr['win'] < -.05)]
player_data_corr.sort_values('win')[['win']]

,win
def_rtg,-0.457735
trb,0.050070
fg,0.060092
ast,0.060231
pts,0.065686
drb,0.065905
fg3,0.066131
fg3_pct,0.096947
fg_pct,0.104209
off_rtg,0.111082


In [7]:
del team_data, team_data_corr, player_data, player_data_corr

In [143]:
features = pd.read_csv('{data_path}/{db_name}.csv'.format(data_path=data_path,
                                                                     db_name=processed_team_data_table_name), sep='|')
features = features[[i for i in features.columns if '_postgame_' not in i]]
features.head()

,team_game_key,team_tag,team_link,team_name,opponent_tag,opponent_link,opponent_name,location,win,year,month,day,date_str,game_key,ast_past_team_data_0_1_records,ast_pct_past_team_data_1_1_records,blk_past_team_data_2_1_records,blk_pct_past_team_data_3_1_records,def_rtg_past_team_data_4_1_records,drb_past_team_data_5_1_records,drb_pct_past_team_data_6_1_records,efg_pct_past_team_data_7_1_records,fg_past_team_data_8_1_records,fg3_past_team_data_9_1_records,fg3_pct_past_team_data_10_1_records,fg3a_past_team_data_11_1_records,fg3a_per_fga_pct_past_team_data_12_1_records,fg_pct_past_team_data_13_1_records,fga_past_team_data_14_1_records,ft_past_team_data_15_1_records,ft_pct_past_team_data_16_1_records,fta_past_team_data_17_1_records,fta_per_fga_pct_past_team_data_18_1_records,mp_past_team_data_19_1_records,off_rtg_past_team_data_20_1_records,orb_past_team_data_21_1_records,orb_pct_past_team_data_22_1_records,pf_past_team_data_23_1_records,plus_minus_past_team_data_24_1_records,pts_past_team_data_25_1_records,stl_past_team_data_26_1_records,stl_pct_past_team_data_27_1_records,tov_past_team_data_28_1_records,tov_pct_past_team_data_29_1_records,trb_past_team_data_30_1_records,trb_pct_past_team_data_31_1_records,ts_pct_past_team_data_32_1_records,usg_pct_past_team_data_33_1_records,feature_home_past_team_data_34_1_records,ast_past_team_data_0_2_records,ast_pct_past_team_data_1_2_records,blk_past_team_data_2_2_records,blk_pct_past_team_data_3_2_records,def_rtg_past_team_data_4_2_records,drb_past_team_data_5_2_records,drb_pct_past_team_data_6_2_records,efg_pct_past_team_data_7_2_records,fg_past_team_data_8_2_records,fg3_past_team_data_9_2_records,fg3_pct_past_team_data_10_2_records,fg3a_past_team_data_11_2_records,fg3a_per_fga_pct_past_team_data_12_2_records,fg_pct_past_team_data_13_2_records,fga_past_team_data_14_2_records,ft_past_team_data_15_2_records,ft_pct_past_team_data_16_2_records,fta_past_team_data_17_2_records,fta_per_fga_pct_past_team_data_18_2_records,mp_past_team_data_19_2_records,off_rtg_past_team_data_20_2_records,orb_past_team_data_21_2_records,orb_pct_past_team_data_22_2_records,pf_past_team_data_23_2_records,plus_minus_past_team_data_24_2_records,pts_past_team_data_25_2_records,stl_past_team_data_26_2_records,stl_pct_past_team_data_27_2_records,tov_past_team_data_28_2_records,tov_pct_past_team_data_29_2_records,trb_past_team_data_30_2_records,trb_pct_past_team_data_31_2_records,ts_pct_past_team_data_32_2_records,usg_pct_past_team_data_33_2_records,feature_home_past_team_data_34_2_records,ast_past_team_data_0_3_records,ast_pct_past_team_data_1_3_records,blk_past_team_data_2_3_records,blk_pct_past_team_data_3_3_records,def_rtg_past_team_data_4_3_records,drb_past_team_data_5_3_records,drb_pct_past_team_data_6_3_records,efg_pct_past_team_data_7_3_records,fg_past_team_data_8_3_records,fg3_past_team_data_9_3_records,fg3_pct_past_team_data_10_3_records,fg3a_past_team_data_11_3_records,fg3a_per_fga_pct_past_team_data_12_3_records,fg_pct_past_team_data_13_3_records,fga_past_team_data_14_3_records,ft_past_team_data_15_3_records,ft_pct_past_team_data_16_3_records,fta_past_team_data_17_3_records,fta_per_fga_pct_past_team_data_18_3_records,mp_past_team_data_19_3_records,off_rtg_past_team_data_20_3_records,orb_past_team_data_21_3_records,orb_pct_past_team_data_22_3_records,pf_past_team_data_23_3_records,plus_minus_past_team_data_24_3_records,pts_past_team_data_25_3_records,stl_past_team_data_26_3_records,stl_pct_past_team_data_27_3_records,tov_past_team_data_28_3_records,tov_pct_past_team_data_29_3_records,trb_past_team_data_30_3_records,trb_pct_past_team_data_31_3_records,ts_pct_past_team_data_32_3_records,usg_pct_past_team_data_33_3_records,feature_home_past_team_data_34_3_records,ast_past_team_data_0_4_records,ast_pct_past_team_data_1_4_records,blk_past_team_data_2_4_records,blk_pct_past_team_data_3_4_records,def_rtg_past_team_data_4_4_records,drb_past_team_data_5_4_records,drb_pct_past_team_data_6_4_records,efg_pct_past_team_dat

In [136]:
# features = pd.read_csv('{data_path}/{db_name}.csv'.format(data_path=data_path,
#                                                                      db_name=combined_feature_file_data_table_name), sep='|')
# features = features[[i for i in features.columns if '_postgame_' not in i]]
# features.head()

,day_player_aggregate_mean,day_player_aggregate_median,day_player_aggregate_amax,day_player_aggregate_amin,day_player_aggregate_var,month_player_aggregate_mean,month_player_aggregate_median,month_player_aggregate_amax,month_player_aggregate_amin,month_player_aggregate_var,year_player_aggregate_mean,year_player_aggregate_median,year_player_aggregate_amax,year_player_aggregate_amin,year_player_aggregate_var,feature_player_past_team_data_pca_dim_0_1_records_player_aggregate_mean,feature_player_past_team_data_pca_dim_0_1_records_player_aggregate_median,feature_player_past_team_data_pca_dim_0_1_records_player_aggregate_amax,feature_player_past_team_data_pca_dim_0_1_records_player_aggregate_amin,feature_player_past_team_data_pca_dim_0_1_records_player_aggregate_var,feature_player_past_team_data_pca_dim_1_1_records_player_aggregate_mean,feature_player_past_team_data_pca_dim_1_1_records_player_aggregate_median,feature_player_past_team_data_pca_dim_1_1_records_player_aggregate_amax,feature_player_past_team_data_pca_dim_1_1_records_player_aggregate_amin,feature_player_past_team_data_pca_dim_1_1_records_player_aggregate_var,feature_player_past_team_data_pca_dim_2_1_records_player_aggregate_mean,feature_player_past_team_data_pca_dim_2_1_records_player_aggregate_median,feature_player_past_team_data_pca_dim_2_1_records_player_aggregate_amax,feature_player_past_team_data_pca_dim_2_1_records_player_aggregate_amin,feature_player_past_team_data_pca_dim_2_1_records_player_aggregate_var,feature_player_past_team_data_pca_dim_3_1_records_player_aggregate_mean,feature_player_past_team_data_pca_dim_3_1_records_player_aggregate_median,feature_player_past_team_data_pca_dim_3_1_records_player_aggregate_amax,feature_player_past_team_data_pca_dim_3_1_records_player_aggregate_amin,feature_player_past_team_data_pca_dim_3_1_records_player_aggregate_var,feature_player_past_team_data_pca_dim_4_1_records_player_aggregate_mean,feature_player_past_team_data_pca_dim_4_1_records_player_aggregate_median,feature_player_past_team_data_pca_dim_4_1_records_player_aggregate_amax,feature_player_past_team_data_pca_dim_4_1_records_player_aggregate_amin,feature_player_past_team_data_pca_dim_4_1_records_player_aggregate_var,feature_player_past_team_data_pca_dim_5_1_records_player_aggregate_mean,feature_player_past_team_data_pca_dim_5_1_records_player_aggregate_median,feature_player_past_team_data_pca_dim_5_1_records_player_aggregate_amax,feature_player_past_team_data_pca_dim_5_1_records_player_aggregate_amin,feature_player_past_team_data_pca_dim_5_1_records_player_aggregate_var,feature_player_past_team_data_pca_dim_6_1_records_player_aggregate_mean,feature_player_past_team_data_pca_dim_6_1_records_player_aggregate_median,feature_player_past_team_data_pca_dim_6_1_records_player_aggregate_amax,feature_player_past_team_data_pca_dim_6_1_records_player_aggregate_amin,feature_player_past_team_data_pca_dim_6_1_records_player_aggregate_var,feature_player_past_team_data_pca_dim_7_1_records_player_aggregate_mean,feature_player_past_team_data_pca_dim_7_1_records_player_aggregate_median,feature_player_past_team_data_pca_dim_7_1_records_player_aggregate_amax,feature_player_past_team_data_pca_dim_7_1_records_player_aggregate_amin,feature_player_past_team_data_pca_dim_7_1_records_player_aggregate_var,feature_player_past_team_data_pca_dim_0_2_records_player_aggregate_mean,feature_player_past_team_data_pca_dim_0_2_records_player_aggregate_median,feature_player_past_team_data_pca_dim_0_2_records_player_aggregate_amax,feature_player_past_team_data_pca_dim_0_2_records_player_aggregate_amin,feature_player_past_team_data_pca_dim_0_2_records_player_aggregate_var,feature_player_past_team_data_pca_dim_1_2_records_player_aggregate_mean,feature_player_past_team_data_pca_dim_1_2_records_player_aggregate_median,feature_player_past_team_data_pca_dim_1_2_records_player_aggregate_amax,feature_player_past_team_data_pca_dim_1_2_records_player_aggregate_amin,feature_player_past_team_data_pca_dim_1_2_records_pl

In [144]:
invalid_columns = {'ast', 'ast_pct', 'blk', 'blk_pct', 'def_rtg', 'drb', 'drb_pct', 'efg_pct',
                                   'fg', 'fg3', 'fg3_pct', 'fg3a', 'fg3a_per_fga_pct', 'fg_pct', 'fga', 'ft', 'ft_pct',
                                   'fta', 'fta_per_fga_pct', 'mp', 'off_rtg', 'orb', 'orb_pct', 'pf',
                                   'plus_minus', 'pts', 'stl', 'stl_pct', 'tov', 'tov_pct', 'trb', 'trb_pct', 'ts_pct',
                                   'usg_pct', 'team_tag', 'team_link', 'team_name', 'opponent_tag', 'opponent_name', 'opponent_link', 'location',
                  'date_str', 'game_key', 'team_game_key', 'temp_col_opponent', 'win_opponent'}
invalid_columns.update({i for i in features.columns if 'postgame' in i})


In [145]:
valid_columns = [i for i in features.columns if i not in invalid_columns | {target}]
for i in valid_columns:
    features[i] = features[i].replace(np.inf, np.nan).replace(-np.inf, np.nan)
    if features[i].isna().sum() / features[i].shape[0] < 1.0:
        features[i] = features[i].fillna(features[i].median())
    features[i] = features[i].fillna(0)

In [146]:

def evaluate_column_vs_target(input_series, target_array, standardization_method):
    # print('input_series: {0}, target_array: {1}, standardization_method:{2}'.format(input_series.shape, target_array.shape, standardization_method))

    input_series = input_series.replace(np.inf, np.nan).replace(-np.inf, np.nan)
    if input_series.isna().sum() / input_series.shape[0] < 1.0:
        input_series = input_series.fillna(input_series.median())
    input_series = input_series.fillna(0)

    if standardization_method:
        transformer = eval(standardization_method)
        input_series = transformer.fit_transform(input_series.values.reshape(-1, 1)).flatten()
    else:
        input_series = input_series.values.reshape(-1, 1).flatten()

    slope, intercept, r_value, p_value, std_err = stats.linregress(input_series, target_array)
    return {'slope':slope,
                'intercept':intercept,
                'r_value':r_value,
                'r2_value':r_value*r_value,
                'p_value':p_value,
                 'std_err':std_err
                }

In [147]:

valid_columns = [i for i in features.columns if i not in invalid_columns | {target}]
interactions = ['product', 'division']
# standardization_methods = [None, 'StandardScaler()']
standardization_methods = [None]
standardization_method = None

target_array = features[target].values

results = list()
print('checking columns')
for counter1, column1 in enumerate(valid_columns):
    print(counter1, column1)
    next_column = features[column1].copy()
    next_results = evaluate_column_vs_target(next_column, target_array, standardization_method)
    next_results.update({'column1':column1,
                         'column2':None,
                         'column3':None,
                'interaction':None})
    results.append(next_results)


            
results_df = pd.DataFrame.from_dict(results)
results_df.head()

checking columns
0 year
1 month
2 day
3 ast_past_team_data_0_1_records
4 ast_pct_past_team_data_1_1_records
5 blk_past_team_data_2_1_records
6 blk_pct_past_team_data_3_1_records
7 def_rtg_past_team_data_4_1_records
8 drb_past_team_data_5_1_records
9 drb_pct_past_team_data_6_1_records
10 efg_pct_past_team_data_7_1_records
11 fg_past_team_data_8_1_records
12 fg3_past_team_data_9_1_records
13 fg3_pct_past_team_data_10_1_records
14 fg3a_past_team_data_11_1_records
15 fg3a_per_fga_pct_past_team_data_12_1_records
16 fg_pct_past_team_data_13_1_records
17 fga_past_team_data_14_1_records
18 ft_past_team_data_15_1_records
19 ft_pct_past_team_data_16_1_records
20 fta_past_team_data_17_1_records
21 fta_per_fga_pct_past_team_data_18_1_records
22 mp_past_team_data_19_1_records
23 off_rtg_past_team_data_20_1_records
24 orb_past_team_data_21_1_records
25 orb_pct_past_team_data_22_1_records
26 pf_past_team_data_23_1_records
27 plus_minus_past_team_data_24_1_records
28 pts_past_team_data_25_1_records
29

264 ft_pct_past_team_data_16_8_records
265 fta_past_team_data_17_8_records
266 fta_per_fga_pct_past_team_data_18_8_records
267 mp_past_team_data_19_8_records
268 off_rtg_past_team_data_20_8_records
269 orb_past_team_data_21_8_records
270 orb_pct_past_team_data_22_8_records
271 pf_past_team_data_23_8_records
272 plus_minus_past_team_data_24_8_records
273 pts_past_team_data_25_8_records
274 stl_past_team_data_26_8_records
275 stl_pct_past_team_data_27_8_records
276 tov_past_team_data_28_8_records
277 tov_pct_past_team_data_29_8_records
278 trb_past_team_data_30_8_records
279 trb_pct_past_team_data_31_8_records
280 ts_pct_past_team_data_32_8_records
281 usg_pct_past_team_data_33_8_records
282 feature_home_past_team_data_34_8_records
283 ast_past_team_data_0_9_records
284 ast_pct_past_team_data_1_9_records
285 blk_past_team_data_2_9_records
286 blk_pct_past_team_data_3_9_records
287 def_rtg_past_team_data_4_9_records
288 drb_past_team_data_5_9_records
289 drb_pct_past_team_data_6_9_records

,slope,intercept,r_value,r2_value,p_value,std_err,column1,column2,column3,interaction
0,0.000000e+00,0.5,0.000000e+00,0.000000e+00,1.0,0.004057,year,None,None,None
1,6.052089e-20,0.5,5.359911e-19,2.872865e-37,1.0,0.001103,month,None,None,None
2,-1.036978e-21,0.5,-1.875105e-20,3.516020e-40,1.0,0.000540,day,None,None,None
3,NaN,NaN,0.000000e+00,0.000000e+00,1.0,inf,ast_past_team_data_0_1_records,None,None,None
4,NaN,NaN,0.000000e+00,0.000000e+00,1.0,inf,ast_pct_past_team_data_1_1_records,None,None,None


In [148]:
results_df = results_df[['column1', 'column2', 'interaction', 'r2_value', 'slope', 'intercept', 'r_value', 'p_value', 'std_err']]

In [149]:
results_df.sort_values('r2_value', ascending = False).head(50)

,column1,column2,interaction,r2_value,slope,intercept,r_value,p_value,std_err
353,feature_team_pregame_rating_0,None,None,4.636943e-02,1.777220e-04,0.313223,2.153356e-01,3.557609e-110,0.000008
1,month,None,None,2.872865e-37,6.052089e-20,0.500000,5.359911e-19,1.000000e+00,0.001103
2,day,None,None,3.516020e-40,-1.036978e-21,0.500000,-1.875105e-20,1.000000e+00,0.000540
352,feature_home_past_team_data_34_10_records,None,None,0.000000e+00,NaN,NaN,0.000000e+00,1.000000e+00,inf
241,tov_past_team_data_28_7_records,None,None,0.000000e+00,NaN,NaN,0.000000e+00,1.000000e+00,inf
240,stl_pct_past_team_data_27_7_records,None,None,0.000000e+00,NaN,NaN,0.000000e+00,1.000000e+00,inf
239,stl_past_team_data_26_7_records,None,None,0.000000e+00,NaN,NaN,0.000000e+00,1.000000e+00,inf
238,pts_past_team_data_25_7_records,None,None,0.000000e+00,NaN,NaN,0.000000e+00,1.000000e+00,inf
237,plus_minus_past_team_data_24_7_records,None,None,0.000000e+00,NaN,NaN,0.000000e+00,1.000000e+00,inf
236,pf_past_team_data_23_7_records,None,None,0.000000e+00,NaN,NaN,0.000000e+00,1.000000e+00,inf


In [129]:

max_iter = 10000

lgbm_params =  {
'boosting_type': 'gbdt',
'objective': 'binary',
'metric': 'binary_error',
"learning_rate": 0.1,
"max_depth": -1,
'num_leaves':31,
}
    

def get_model_score(x, y):
    train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=.3333, random_state=1)
    train_x, val_x, train_y, val_y = train_test_split(x, y, test_size=.5, random_state=1)
    lgtrain = lgb.Dataset(train_x, train_y)
    lgvalid = lgb.Dataset(val_x, val_y)

    model = lgb.train(
        lgbm_params,
        lgtrain,
        num_boost_round=max_iter,
        valid_sets=[lgtrain, lgvalid],
        valid_names=['train', 'valid'],
        early_stopping_rounds=10,
        verbose_eval=0
    )
    return accuracy_score(test_y, np.rint(model.predict(test_x, num_iteration=model.best_iteration)))
    

    


In [130]:
features.shape

(8274, 6834)

In [131]:
get_model_score(features[valid_columns], features[target])

0.6443074691805656

In [132]:
    
possible_interactions = ['product', 'division']

def add_best_feature(features, starting_columns, interaction_list, interactions = 10000):
    
    min_score = get_model_score(features[starting_columns], features[target])
    valid_columns = [i for i in features.columns if i not in invalid_columns | {target}]
    
    for i in valid_columns:
        features[i] = features[i].replace(np.inf, np.nan).replace(-np.inf, np.nan)
        if features[i].isna().sum() / features[i].shape[0] < 1.0:
            features[i] = features[i].fillna(features[i].median())
        features[i] = features[i].fillna(0)

    results = list()
    for i in tqdm.tqdm(valid_columns):
        score = get_model_score(features[starting_columns + [i]], features[target])
        results.append({'column1':i,
                               'column2':None,
                               'interaction':None,
                               'score':score})
    
    if interactions:
        new_interaction_list = list()
        
        for i in valid_columns:
            for j in valid_columns:
                for k in possible_interactions:
                    new_interaction_list.append({'column1':i,
                                   'column2':j,
                                   'interaction':k})
        new_interaction_list = random.sample(new_interaction_list, interactions)
        
        for i in tqdm.tqdm(new_interaction_list):
            if i['interaction'] == 'product':
                interaction_list
                input_series = features[i['column1']] * features[i['column2']]
            if i['interaction']== 'division':
                input_series = features[i['column1']] / features[i['column2']]

            input_series = input_series.replace(np.inf, np.nan).replace(-np.inf, np.nan)
            if input_series.isna().sum() / input_series.shape[0] < 1.0:
                input_series = input_series.fillna(input_series.median())
            input_series = input_series.fillna(0)
            features['temp_column'] = input_series

            score = get_model_score(features[starting_columns + ['temp_column']], features[target])
            print(score)
            results.append({'column1':i['column1'],
                           'column2':i['column2'],
                           'interaction':i['interaction'],
                           'score':score})

    results = sorted(results, key = lambda x: x['score'], reverse = True)
    print(results[0:1])
    
    for i in results:
        if i['interaction'] == 'product':
            input_series = features[i['column1']] * features[i['column2']]
            col_name = '{0}_product_{1}'.format(i['column1'], i['column2'])
        if i['interaction'] == 'division':
            input_series = features[i['column1']] / features[i['column2']]
            col_name = '{0}_division_{1}'.format(i['column1'], i['column2'])
        if not i['interaction']:
            col_name = i['column1']
        if i['interaction']:
            input_series = input_series.replace(np.inf, np.nan).replace(-np.inf, np.nan)
            if input_series.isna().sum() / input_series.shape[0] < 1.0:
                input_series = input_series.fillna(combined_column.median())
            input_series = input_series.fillna(0)
            features[col_name] = input_series
            
        if i not in interaction_list and col_name not in starting_columns and i['score'] > min_score:
            starting_columns.append(col_name)
            interaction_list.append(i)
            break
            
    return features, starting_columns, interaction_list
    


In [133]:

    
    # features[i] = StandardScaler().fit_transform( features[i].values.reshape(-1, 1)).flatten()

In [134]:
features.describe()

,day_player_aggregate_mean,day_player_aggregate_median,day_player_aggregate_amax,day_player_aggregate_amin,day_player_aggregate_var,month_player_aggregate_mean,month_player_aggregate_median,month_player_aggregate_amax,month_player_aggregate_amin,month_player_aggregate_var,year_player_aggregate_mean,year_player_aggregate_median,year_player_aggregate_amax,year_player_aggregate_amin,year_player_aggregate_var,feature_player_rl_avg_ast_1_player_aggregate_mean,feature_player_rl_avg_ast_1_player_aggregate_median,feature_player_rl_avg_ast_1_player_aggregate_amax,feature_player_rl_avg_ast_1_player_aggregate_amin,feature_player_rl_avg_ast_1_player_aggregate_var,feature_player_rl_avg_ast_2_player_aggregate_mean,feature_player_rl_avg_ast_2_player_aggregate_median,feature_player_rl_avg_ast_2_player_aggregate_amax,feature_player_rl_avg_ast_2_player_aggregate_amin,feature_player_rl_avg_ast_2_player_aggregate_var,feature_player_rl_avg_ast_3_player_aggregate_mean,feature_player_rl_avg_ast_3_player_aggregate_median,feature_player_rl_avg_ast_3_player_aggregate_amax,feature_player_rl_avg_ast_3_player_aggregate_amin,feature_player_rl_avg_ast_3_player_aggregate_var,feature_player_rl_avg_ast_4_player_aggregate_mean,feature_player_rl_avg_ast_4_player_aggregate_median,feature_player_rl_avg_ast_4_player_aggregate_amax,feature_player_rl_avg_ast_4_player_aggregate_amin,feature_player_rl_avg_ast_4_player_aggregate_var,feature_player_rl_avg_ast_5_player_aggregate_mean,feature_player_rl_avg_ast_5_player_aggregate_median,feature_player_rl_avg_ast_5_player_aggregate_amax,feature_player_rl_avg_ast_5_player_aggregate_amin,feature_player_rl_avg_ast_5_player_aggregate_var,feature_player_rl_avg_ast_50_player_aggregate_mean,feature_player_rl_avg_ast_50_player_aggregate_median,feature_player_rl_avg_ast_50_player_aggregate_amax,feature_player_rl_avg_ast_50_player_aggregate_amin,feature_player_rl_avg_ast_50_player_aggregate_var,feature_player_rl_avg_ast_pct_1_player_aggregate_mean,feature_player_rl_avg_ast_pct_1_player_aggregate_median,feature_player_rl_avg_ast_pct_1_player_aggregate_amax,feature_player_rl_avg_ast_pct_1_player_aggregate_amin,feature_player_rl_avg_ast_pct_1_player_aggregate_var,feature_player_rl_avg_ast_pct_2_player_aggregate_mean,feature_player_rl_avg_ast_pct_2_player_aggregate_median,feature_player_rl_avg_ast_pct_2_player_aggregate_amax,feature_player_rl_avg_ast_pct_2_player_aggregate_amin,feature_player_rl_avg_ast_pct_2_player_aggregate_var,feature_player_rl_avg_ast_pct_3_player_aggregate_mean,feature_player_rl_avg_ast_pct_3_player_aggregate_median,feature_player_rl_avg_ast_pct_3_player_aggregate_amax,feature_player_rl_avg_ast_pct_3_player_aggregate_amin,feature_player_rl_avg_ast_pct_3_player_aggregate_var,feature_player_rl_avg_ast_pct_4_player_aggregate_mean,feature_player_rl_avg_ast_pct_4_player_aggregate_median,feature_player_rl_avg_ast_pct_4_player_aggregate_amax,feature_player_rl_avg_ast_pct_4_player_aggregate_amin,feature_player_rl_avg_ast_pct_4_player_aggregate_var,feature_player_rl_avg_ast_pct_5_player_aggregate_mean,feature_player_rl_avg_ast_pct_5_player_aggregate_median,feature_player_rl_avg_ast_pct_5_player_aggregate_amax,feature_player_rl_avg_ast_pct_5_player_aggregate_amin,feature_player_rl_avg_ast_pct_5_player_aggregate_var,feature_player_rl_avg_ast_pct_50_player_aggregate_mean,feature_player_rl_avg_ast_pct_50_player_aggregate_median,feature_player_rl_avg_ast_pct_50_player_aggregate_amax,feature_player_rl_avg_ast_pct_50_player_aggregate_amin,feature_player_rl_avg_ast_pct_50_player_aggregate_var,feature_player_rl_avg_blk_1_player_aggregate_mean,feature_player_rl_avg_blk_1_player_aggregate_median,feature_player_rl_avg_blk_1_player_aggregate_amax,feature_player_rl_avg_blk_1_player_aggregate_amin,feature_player_rl_avg_blk_1_player_aggregate_var,feature_player_rl_avg_blk_2_player_aggregate_mean,feature_player_rl_avg_blk_2_player_aggregate_median,feature_player_rl_avg_blk_2_player_aggregate_amax,feature_player_rl_avg_blk_2_player_aggregate_a

In [135]:
current_columns = ['feature_team_pregame_rating_0_difference_over_opponent']
interaction_list = list()

while len(current_columns) < 24:
    features, current_columns, interaction_list= add_best_feature(features, current_columns, interaction_list, interactions = 0)
    print(current_columns, interaction_list)
current_columns








  0%|          | 0/6831 [00:00<?, ?it/s]






  0%|          | 3/6831 [00:00<04:11, 27.15it/s]






  0%|          | 7/6831 [00:00<03:53, 29.18it/s]






  0%|          | 11/6831 [00:00<03:39, 31.09it/s]






  0%|          | 15/6831 [00:00<03:38, 31.22it/s]






  0%|          | 18/6831 [00:00<03:41, 30.69it/s]






  0%|          | 22/6831 [00:00<03:35, 31.61it/s]






  0%|          | 26/6831 [00:00<03:24, 33.29it/s]






  0%|          | 30/6831 [00:00<03:29, 32.49it/s]






  0%|          | 34/6831 [00:01<03:51, 29.33it/s]






  1%|          | 37/6831 [00:01<04:11, 27.05it/s]






  1%|          | 40/6831 [00:01<04:04, 27.76it/s]






  1%|          | 43/6831 [00:01<04:35, 24.66it/s]






  1%|          | 47/6831 [00:01<04:46, 23.71it/s]






  1%|          | 50/6831 [00:01<04:48, 23.50it/s]






  1%|          | 54/6831 [00:01<04:22, 25.79it/s]






  1%|          | 58/6831 [00:02<03:54, 28.83it/s]






  1%|          | 62/6831 [00:02<05:28, 20.63it/s]




  6%|▌         | 385/6831 [00:16<03:10, 33.78it/s]






  6%|▌         | 389/6831 [00:17<06:31, 16.47it/s]






  6%|▌         | 392/6831 [00:17<08:14, 13.02it/s]






  6%|▌         | 395/6831 [00:17<09:58, 10.75it/s]






  6%|▌         | 397/6831 [00:17<09:56, 10.79it/s]






  6%|▌         | 399/6831 [00:18<10:32, 10.17it/s]






  6%|▌         | 401/6831 [00:18<10:58,  9.77it/s]






  6%|▌         | 403/6831 [00:18<09:43, 11.01it/s]






  6%|▌         | 407/6831 [00:18<07:40, 13.96it/s]






  6%|▌         | 411/6831 [00:18<06:15, 17.09it/s]






  6%|▌         | 415/6831 [00:18<05:18, 20.17it/s]






  6%|▌         | 419/6831 [00:19<04:36, 23.20it/s]






  6%|▌         | 423/6831 [00:19<04:18, 24.78it/s]






  6%|▋         | 427/6831 [00:19<03:58, 26.85it/s]






  6%|▋         | 431/6831 [00:19<03:38, 29.24it/s]






  6%|▋         | 435/6831 [00:19<03:26, 30.91it/s]






  6%|▋         | 440/6831 [00:19<03:09, 33.77it/s]






  6%|▋         | 444/6831 [00:1

 11%|█         | 767/6831 [00:34<03:28, 29.07it/s]






 11%|█▏        | 771/6831 [00:34<03:13, 31.26it/s]






 11%|█▏        | 775/6831 [00:34<03:07, 32.29it/s]






 11%|█▏        | 779/6831 [00:34<03:00, 33.57it/s]






 11%|█▏        | 783/6831 [00:34<04:58, 20.28it/s]






 12%|█▏        | 786/6831 [00:34<05:52, 17.16it/s]






 12%|█▏        | 789/6831 [00:35<06:53, 14.60it/s]






 12%|█▏        | 791/6831 [00:35<08:28, 11.87it/s]






 12%|█▏        | 793/6831 [00:35<08:57, 11.22it/s]






 12%|█▏        | 795/6831 [00:35<09:27, 10.63it/s]






 12%|█▏        | 797/6831 [00:36<10:45,  9.35it/s]






 12%|█▏        | 799/6831 [00:36<09:42, 10.36it/s]






 12%|█▏        | 804/6831 [00:36<07:27, 13.47it/s]






 12%|█▏        | 809/6831 [00:36<05:51, 17.12it/s]






 12%|█▏        | 814/6831 [00:36<04:49, 20.78it/s]






 12%|█▏        | 819/6831 [00:36<04:05, 24.45it/s]






 12%|█▏        | 823/6831 [00:36<03:39, 27.34it/s]






 12%|█▏        | 828/6831 [00:3

 17%|█▋        | 1138/6831 [00:52<07:15, 13.08it/s]






 17%|█▋        | 1142/6831 [00:52<05:52, 16.13it/s]






 17%|█▋        | 1146/6831 [00:52<04:51, 19.48it/s]






 17%|█▋        | 1150/6831 [00:52<04:10, 22.72it/s]






 17%|█▋        | 1155/6831 [00:52<03:37, 26.07it/s]






 17%|█▋        | 1159/6831 [00:52<03:20, 28.25it/s]






 17%|█▋        | 1163/6831 [00:53<03:53, 24.22it/s]






 17%|█▋        | 1167/6831 [00:53<03:29, 27.02it/s]






 17%|█▋        | 1171/6831 [00:53<03:11, 29.59it/s]






 17%|█▋        | 1175/6831 [00:53<02:56, 32.07it/s]






 17%|█▋        | 1179/6831 [00:53<02:58, 31.67it/s]






 17%|█▋        | 1184/6831 [00:53<02:44, 34.34it/s]






 17%|█▋        | 1188/6831 [00:53<03:23, 27.74it/s]






 17%|█▋        | 1192/6831 [00:54<03:13, 29.19it/s]






 18%|█▊        | 1196/6831 [00:54<03:07, 30.06it/s]






 18%|█▊        | 1201/6831 [00:54<03:30, 26.76it/s]






 18%|█▊        | 1205/6831 [00:54<03:11, 29.32it/s]






 18%|█▊       

 22%|██▏       | 1503/6831 [01:09<11:56,  7.43it/s]






 22%|██▏       | 1506/6831 [01:09<09:16,  9.57it/s]






 22%|██▏       | 1511/6831 [01:09<07:07, 12.44it/s]






 22%|██▏       | 1515/6831 [01:09<05:42, 15.54it/s]






 22%|██▏       | 1519/6831 [01:09<04:40, 18.94it/s]






 22%|██▏       | 1523/6831 [01:09<04:00, 22.04it/s]






 22%|██▏       | 1527/6831 [01:09<03:31, 25.11it/s]






 22%|██▏       | 1531/6831 [01:09<03:09, 27.96it/s]






 22%|██▏       | 1535/6831 [01:10<02:57, 29.79it/s]






 23%|██▎       | 1540/6831 [01:10<02:42, 32.58it/s]






 23%|██▎       | 1544/6831 [01:10<02:33, 34.41it/s]






 23%|██▎       | 1548/6831 [01:10<02:38, 33.41it/s]






 23%|██▎       | 1552/6831 [01:10<02:33, 34.37it/s]






 23%|██▎       | 1556/6831 [01:10<02:28, 35.43it/s]






 23%|██▎       | 1560/6831 [01:10<02:35, 33.91it/s]






 23%|██▎       | 1565/6831 [01:10<02:25, 36.28it/s]






 23%|██▎       | 1569/6831 [01:10<02:23, 36.64it/s]






 23%|██▎      

...

 74%|███████▎  | 5025/6831 [06:14<05:06,  5.90it/s]






 74%|███████▎  | 5027/6831 [06:14<04:46,  6.30it/s]






 74%|███████▎  | 5028/6831 [06:14<05:09,  5.83it/s]






 74%|███████▎  | 5029/6831 [06:14<05:03,  5.93it/s]






 74%|███████▎  | 5030/6831 [06:14<04:50,  6.21it/s]






 74%|███████▎  | 5031/6831 [06:14<04:25,  6.79it/s]






 74%|███████▎  | 5032/6831 [06:15<05:42,  5.24it/s]






 74%|███████▎  | 5033/6831 [06:15<06:16,  4.77it/s]






 74%|███████▎  | 5034/6831 [06:15<07:05,  4.22it/s]






 74%|███████▎  | 5035/6831 [06:15<07:00,  4.27it/s]






 74%|███████▎  | 5036/6831 [06:16<07:16,  4.11it/s]






 74%|███████▎  | 5037/6831 [06:16<07:06,  4.21it/s]






 74%|███████▍  | 5038/6831 [06:16<07:18,  4.09it/s]






 74%|███████▍  | 5039/6831 [06:17<07:39,  3.90it/s]






 74%|███████▍  | 5040/6831 [06:17<07:26,  4.01it/s]






 74%|███████▍  | 5041/6831 [06:17<06:52,  4.34it/s]






 74%|███████▍  | 5042/6831 [06:17<06:21,  4.68it/s]






 74%|███████▍ 

 76%|███████▋  | 5223/6831 [06:33<02:34, 10.43it/s]






 77%|███████▋  | 5227/6831 [06:34<02:03, 13.00it/s]






 77%|███████▋  | 5231/6831 [06:34<01:41, 15.75it/s]






 77%|███████▋  | 5234/6831 [06:34<01:31, 17.51it/s]






 77%|███████▋  | 5237/6831 [06:34<01:20, 19.88it/s]






 77%|███████▋  | 5240/6831 [06:34<01:17, 20.58it/s]






 77%|███████▋  | 5243/6831 [06:34<01:14, 21.24it/s]






 77%|███████▋  | 5246/6831 [06:34<01:18, 20.24it/s]






 77%|███████▋  | 5249/6831 [06:34<01:11, 22.23it/s]






 77%|███████▋  | 5252/6831 [06:35<01:07, 23.26it/s]






 77%|███████▋  | 5255/6831 [06:35<01:04, 24.30it/s]






 77%|███████▋  | 5258/6831 [06:35<01:04, 24.26it/s]






 77%|███████▋  | 5261/6831 [06:35<01:14, 21.19it/s]






 77%|███████▋  | 5264/6831 [06:35<01:24, 18.52it/s]






 77%|███████▋  | 5267/6831 [06:35<01:38, 15.83it/s]






 77%|███████▋  | 5269/6831 [06:36<01:49, 14.33it/s]






 77%|███████▋  | 5271/6831 [06:36<03:52,  6.70it/s]






 77%|███████▋ 

 79%|███████▉  | 5415/6831 [06:54<04:23,  5.37it/s]






 79%|███████▉  | 5416/6831 [06:54<03:50,  6.14it/s]






 79%|███████▉  | 5417/6831 [06:54<04:19,  5.45it/s]






 79%|███████▉  | 5418/6831 [06:54<04:13,  5.58it/s]






 79%|███████▉  | 5420/6831 [06:55<04:00,  5.86it/s]






 79%|███████▉  | 5422/6831 [06:55<03:32,  6.62it/s]






 79%|███████▉  | 5423/6831 [06:55<03:25,  6.86it/s]






 79%|███████▉  | 5424/6831 [06:55<03:39,  6.41it/s]






 79%|███████▉  | 5425/6831 [06:55<03:57,  5.93it/s]






 79%|███████▉  | 5426/6831 [06:55<03:58,  5.89it/s]






 79%|███████▉  | 5427/6831 [06:56<04:05,  5.73it/s]






 79%|███████▉  | 5428/6831 [06:56<04:09,  5.62it/s]






 79%|███████▉  | 5429/6831 [06:56<04:47,  4.87it/s]






 79%|███████▉  | 5430/6831 [06:56<05:14,  4.46it/s]






 80%|███████▉  | 5431/6831 [06:57<05:31,  4.23it/s]






 80%|███████▉  | 5433/6831 [06:57<04:49,  4.83it/s]






 80%|███████▉  | 5434/6831 [06:57<04:17,  5.42it/s]






 80%|███████▉ 

 82%|████████▏ | 5634/6831 [07:13<00:59, 20.12it/s]






 83%|████████▎ | 5637/6831 [07:13<00:54, 21.91it/s]






 83%|████████▎ | 5640/6831 [07:13<00:51, 22.93it/s]






 83%|████████▎ | 5644/6831 [07:13<00:48, 24.54it/s]






 83%|████████▎ | 5647/6831 [07:14<00:48, 24.41it/s]






 83%|████████▎ | 5650/6831 [07:14<00:48, 24.25it/s]






 83%|████████▎ | 5653/6831 [07:14<00:47, 24.60it/s]






 83%|████████▎ | 5656/6831 [07:14<00:51, 23.03it/s]






 83%|████████▎ | 5659/6831 [07:14<01:07, 17.29it/s]






 83%|████████▎ | 5661/6831 [07:14<01:22, 14.20it/s]






 83%|████████▎ | 5663/6831 [07:15<02:32,  7.68it/s]






 83%|████████▎ | 5665/6831 [07:16<04:32,  4.27it/s]






 83%|████████▎ | 5666/6831 [07:16<04:13,  4.59it/s]






 83%|████████▎ | 5667/6831 [07:16<03:54,  4.97it/s]






 83%|████████▎ | 5668/6831 [07:16<03:55,  4.94it/s]






 83%|████████▎ | 5669/6831 [07:17<04:37,  4.18it/s]






 83%|████████▎ | 5670/6831 [07:17<04:17,  4.51it/s]






 83%|████████▎

KeyboardInterrupt: 

In [ ]:
while len(current_columns) < 100:
    features, current_columns, interaction_list= add_best_feature(features, current_columns, interaction_list, interactions = 100000)
    print(current_columns, interaction_list)
current_columns